In [1]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.multiclass import OneVsRestClassifier
from xgboost import plot_tree
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import multilabel_confusion_matrix

import json
import glob
import os
import numpy as np
from matplotlib import pyplot as plt

In [2]:
loadFromMerged=True
loadFromIndexes= False
Mapper='SE'
IgnoreEmpty= True
FoldID ="1"
Epoch_count=100
Batch_size=5000


valid_event_values = set(['on','off','open','closed','locked','unlocked','XXX'])


In [3]:
# %run './utils.ipynb'

In [4]:
def __print_info(y_test, pred , classes , confidence=0.90 ):
    # print(classification_report(y_test, pred, target_names=classes))
    # print(hamming_loss(y_test,pred))
    
    
    def make_recall_shit( inp ):
        tp = inp[1][1]
        tn = inp[0][0]
        fp = inp[0][1] 
        fn = inp[1][0]
        
        acc = (tp+tn)*1.0 / ( tp+tn+fp+fn)*1.0
        recall = tp*1.0/ ( tp+fn ) *1.0
        prec = tp*1.0 / ( tp+fp )*1.0
        
    #   F= 2.0*( prec* recall )/ (prec+recall)
        F= 2.0*( tp)/ (2*tp + fp + fn)
        
        return acc, recall, prec, F

    counts = np.sum( y_test.astype(int) , axis=0)
    
    if confidence<0.90:
        pred[pred>=confidence] = 1
        pred[pred<confidence] = 0
    
#     from sklearn.metrics import f1_score
#     f1 = f1_score(y_test, pred, average='micro')
#     print("Micro F1 : %f" % f1)
    
    
    
    def get_cm_count():
        l = len(y_test[0])
        cm_count = np.zeros((l,l))
        for i,y in enumerate(y_test):   
            right = np.bitwise_and(y.astype(int),pred[i].astype(int))  
            if len(np.argwhere(right==1))>0:
                for x in np.argwhere(right==1)[0]:
                
                    cm_count[x][x] +=1 
            wrong = np.bitwise_xor(y.astype(int),pred[i].astype(int))
            # print('---')
            # print(np.argwhere(y==1), np.argwhere(pred[i]==1)  )  
            if len(np.argwhere(wrong==1))>0:
                # print('====>>>')
                # print(np.argwhere(wrong==1))
                if len(np.argwhere(y==1))>0:
                    for x in np.argwhere(y==1)[0]:
                        for w in np.argwhere(wrong==1)[0]:    
                                                
                            cm_count[x][w] +=1 if x!=w   else 0
                            # cm_count[w][x] +=1 if x!=w   else 0
                    # for x in np.argwhere(y==0)[0]:
                    #     cm_count[w][x] +=1 
        return cm_count
            # break
    
    #   acc_wierd  =acc_match_wierd(y_test, pred)   
    def plot_confusion_matrix(cm, classes,
                          normalize=True,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues, 
                         showNumbers = False):
        """
        This function prints and plots the confusion matrix.
        Normalization can be applied by setting `normalize=True`.
        """
        if normalize:
            cm = cm.astype('float') / cm.max(axis=1)[:, np.newaxis]
            print("Normalized confusion matrix")
        else:
            print('Confusion matrix, without normalization')

        # print(cm)
        plt.figure( figsize=(16,16) )
        # plt.tick_params(labelsize=10) 
        plt.imshow(cm, interpolation='nearest', cmap=cmap )
        
        plt.rcParams.update({'font.size': 22}) 
        plt.title(title,size=30)
        plt.colorbar()
        plt.tick_params(axis='both',  labelsize=18)
        tick_marks = np.arange(len(classes))
        plt.xticks(tick_marks, classes, rotation=270)
        plt.yticks(tick_marks, classes)

        fmt = '.2f' if normalize else '.0f'
        thresh = cm.max() / 2.
        if showNumbers:
            for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
                plt.text(j, i, format(cm[i, j], fmt),
                        horizontalalignment="center",
                        color="white" if cm[i, j] > thresh else "black")
        plt.show()

    conf = multilabel_confusion_matrix( y_test , pred.astype(int), labels= range(len(classes)))
    # plot_confusion_matrix(get_cm_count(), classes)
    # conf= multilabel_confusion_matrix( y_test , pred.astype(int), labels= range(len(classes)))
    accs = [make_recall_shit(x) for x in conf]
    print( "%30s  %8s   %8s  %8s  %8s %8s %22s" %( "Class","Accuracy", "Recall","Precision","F Score" , "Count", "TP/TN/FP/FN"))
    print( "------------------------------------------------------------------------" )
    
    for index in range(len(classes)):
        # print(classes[index], conf[index])
        tp = conf[index][1][1]
        tn = conf[index][0][0]
        fp = conf[index][0][1] 
        fn = conf[index][1][0]
        
        print( "%30s  %8.3f   %8.3f  %8.3f  %8.3f  %8d %5d/%5d/%5d/%5d"  %
             (classes[index],
              accs[index][0],
              accs[index][1],
              accs[index][2],
              accs[index][3],
              counts[index],
                  tp ,
                tn ,
                fp ,
                fn ))
    n_zeros_true = len([ x  for x  in  [np.sum(np.abs( y_test[i] )) for i in range(len(y_test))] if x  == 0]  )
    n_zeros_pred = len([ x  for x  in  [np.sum(np.abs( pred[i] )) for i in range(len(pred))] if x  == 0]  )
    
    accs = np.nan_to_num(accs)
    
    def acc_match( true, pred ):
        """
        returns exact mathc accuracy
        """
        
        return (len( [ x  for x  in  [np.sum(np.abs( true[i]- pred[i] )) for i in range(len(true))] if x  == 0]))*1.0 / len(true)
    
    def acc_match_wierd( true, pred ):
        """
        returns exact mathc accuracy
        """
        level = 6 
        switch = 11
        threeAxis=13
        accel = 0 
        status=10
        contact=5

        counter  = 0 
        for i in range( len (true) ):
            if np.sum(np.abs( true[i]- pred[i] ))==0 :
                counter+=1
            else : 
                t_rec = np.array(list( pred[i]))
                
                if true[i][level]==1 or true[i][switch]==1 or t_rec[level]==1 :
                    t_rec[switch]=1
                
                if true[i][threeAxis]==1 or true[i][accel]==1 or t_rec[threeAxis]==1:
                    t_rec[accel] =1
                
                if true[i][status]==1 or true[i][contact]==1 or t_rec[status]==1:
                    t_rec[contact]=1
            #             print(t_rec , true[i])    
                if np.sum(np.abs( true[i]- t_rec ))==0 :
                    counter+=1   
        return counter*1.0 / len(true)

    print ("------------------------------------------------------------------------")
    print( "%30s  %8.3f   %8.3f  %8.3f  %8.3f  %8d %5d/%5d/%5d/%5d"  %
             ("AVERAGES",
              np.average( accs, axis=0)[0],
              np.average( accs, axis=0)[1],
              np.average( accs, axis=0)[2],
              np.average( accs, axis=0)[3],
              len(y_test),
                  0 ,
                0,
                0 ,
                0 ))
    print ( "Exact Match ACC : %.5f " % acc_match( y_test, pred )  )

    # print ( "Wierd Exact Match ACC : %.5f" % acc_wierd)
    print ( "Total Records : %d " % len(y_test)  )
    print ( "Total ZXeros in True : %d (%.3f)%%" % (n_zeros_true ,  n_zeros_true * 1.0/ len(y_test)  ))
    print ( "Total ZXeros in Test : %d (%.3f)%%" % (n_zeros_pred ,  n_zeros_pred * 1.0/ len(y_test)  ) )
    print ('=============================================================================')


In [5]:
def clean_data( x_data, y_data , removeempty=True, Mapping='S'):

    #  mapps the input records to a integer array for the input
    def mapping_x( inp, includeDirection = False , TrimAt= 15 ):
        includeDirection=True
        if includeDirection:
            return np.array([ x["frame_length"] + 
            ('+' if x['packet_source']=='hub' else '-')  
            for x in inp ])
        else:
            # return np.array([ int(int(x["frame_length"])/10)*10  for x in inp ])
            return np.array([ x["frame_length"]  for x in inp ])

    def mapping_y_service(inp):
        return np.array(  list(set([x["event"] for x in inp])) if (len(inp )>0) else ["none"] )

    def mapping_y_service_event(inp):
        return np.array(  list(set([ ("%s-%s"%( x["event"] ,x["val"] )).replace('unknown-','unknown') for x in inp])) if (len(inp )>0) else ["none"] )

    def mapping_y_device_service(inp):
        return np.array(  list(set([ "%s & %s"%( x["device"] ,x["event"] ) for x in inp])) if (len(inp )>0) else ["none"] )

    def mapping_y_full(inp):
        return np.array(  list(set([ "%s & %s & %s"%( x["device"] ,x["event"], x['val'] ) for x in inp])) if (len(inp )>0) else ["none"] )

    # print(y_data)
    cleans = [] 
    cleans = (sorted([ x for x in y_data if  len(y_data[x]) > 0 ] ))
    
    ret_x_data  = [x_data[x] for x in cleans]
    ret_y_data  = [y_data[x] for x in cleans] 
    
    # print( len(y_data), len(cleans) )
    # Mapping='SE'
    ret_x  = [ mapping_x(x) for x in ret_x_data ] 
    ret_y_s = [ mapping_y_service(y) for y in ret_y_data ]
    if Mapping=='S':
        ret_y  = [ mapping_y_service(y) for y in ret_y_data ]
    elif Mapping=='SE':
        ret_y  = [ mapping_y_service_event(y) for y in ret_y_data ]
    elif Mapping=='DS':
        ret_y  = [ mapping_y_device_service(y) for y in ret_y_data ]
    elif Mapping=='F':
        ret_y  = [ mapping_y_full(y) for y in ret_y_data ]
    return ret_x, ret_y, ret_x_data


In [6]:
def pre_process_raw( x_data,y_data, dim_size = 128, test = False, normalize = False ,classes=None, twoD= False, string =False ):
    #  y data 
    # """
    # this functino is in charge of preprocessing the records , the sourc e json contains a lot of extra stuff, this function tailors
    # the data and it fixes their lenghth
    # """
    global terms_dic
    # print(x_data)

    if classes is None:
        classes  = sorted(list(np.unique(  np.concatenate( y_data  ))))
    else :
        classes = sorted(classes)
    if classes and 'unknown' not in classes:
        classes.append('unknown') 
    classes = sorted(classes)  
    services=classes
    y_data_categorical = []  
    x_data_temp = [] 
    y_class_data = []
    print(classes)
    classes_count={}
    for i,labels in enumerate(y_data):
        classFound=False
        temp = np.zeros( len(classes) )
        # print(x)
        y_class=[]
        
        # if 'lock' in labels  and 'known' not in classes and test:
        #     print(labels)
        for label in labels :
            if label=='unknown' and len(labels)>1:
                continue
            if label in classes and (not test or classes_count.get(label,0)<150):
                y_class.append(label)
                temp[ classes.index( label ) ] = 1
                if label!='unknown' and label!='ping':
                    classes_count[label] = classes_count.get(label,0)+1
                classFound=True #if not classFound else classFound
                # if 'lock' in label and 'known' not in classes and test:
                #     print(label,classFound)

        
        
        if 'known' in classes and not classFound:
            temp[ classes.index( 'known') ] = 1                                
            y_class = labels
                         
        elif not classFound and not test:
            temp[ classes.index('unknown') ] = 1
            y_class.append('unknown')
            # print(y_class)
        
        

        if terms_dic and not test:          
            x_data_valid = np.array([seq for seq in x_data[i] if any([seq.split()[0] in terms_dic[lbl]for lbl in labels])])
        else:      
            x_data_valid=x_data[i]   
       
        if len(x_data_valid)>0 and len(y_class)>0:
            y_data_categorical.append( temp ) 
            # print(x_data_valid)
            x_data_temp.append(x_data_valid)
            y_class_data.append(' '.join(y_class))
            # if 'known' not in classes and test and 'lock' in labels:
            #     print(classFound,y_class,labels,x_data_valid)

    # print(x_data_temp)
    # print(y_data_categorical    )
    y_data_categorical = np.vstack(y_data_categorical)
    # print(y_data_categorical)
    x_data=x_data_temp
    # print(x_data)
    x_data_temp=[]
    temp = [] 
    
    for x in x_data:
        temp = [] #list(x)
        lst = list(x)
        temp = lst
        while len(temp)<dim_size:
            temp.extend(lst[:dim_size-len(lst)])
        # while dim_size - len(temp )   > len(lst):
        #     temp.extend(lst)

        # while len(temp) < dim_size:
        #     temp.append( 0 )
        if not string:
            x_data_temp.append(np.array(temp))
        else:
            x_data_temp.append(' '.join(list(map(str,temp))))
    #     # print(temp)
    # print(x_data)
    
    if normalize:
        x_data_temp = np.array( x_data_temp) / (np.amax( x_data_temp) + 0.000000000001)
    else :
        x_data_temp = np.array(x_data_temp)
    
    print(x_data_temp.shape)
   
    return  x_data_temp ,y_data_categorical , y_class_data



In [7]:
x_data= []
y_data= []
x_test = []
y_test = []
y_test_service= []
x_train = []
y_train = []
test_names = []
y_train_service=[]
y_train_process_encoded=[]

test_names = []

add_to_trainig = [0,2]

if loadFromMerged:
    print("in load from merged")
    for pick in sorted(glob.glob( '../files/iot_data_2020/train/hub_segments/*.json' )):
        fname  = os.path.basename(pick)
#             test_names.append( fname )
        with open( os.path.join( '../files/iot_data_2020/train/hub_segments/', fname) ) as f:
            y_data = json.load(f)

        with open( os.path.join('../files/iot_data_2020/train/pcap_segments/', fname) ) as f:
            x_data = json.load(f)

        if len( y_data ) != len(x_data) :
            print( pick )
            continue

        x_t,y_t, y_t_s= clean_data( x_data, y_data, IgnoreEmpty , 
                                                 Mapping=Mapper )

        x_train.extend(x_t)
        y_train.extend(y_t)
        y_train_service.extend(y_t_s)

    print("loading from test files")
    files_path =  '../files/iot_data_2020/usecases/pcap_segments_final/*.json'
    test_y_dir = '../files/iot_data_2020/usecases/hub_segments_final/'
    test_x_dir =  '../files/iot_data_2020/usecases/pcap_segments_final/'
    
    test_files = sorted(glob.glob(files_path))
    print( "found files : " , len(test_files) )
    
    
    for pick  in test_files:
        if 'lock' in pick:
            continue
        fname  = os.path.basename(pick)
        test_names.append( fname )
        print(fname)
        with open( os.path.join(test_y_dir , fname) ) as f:
            y_data_test = json.load(f)

        with open( os.path.join(test_x_dir, fname) ) as f:
            x_data_test = json.load(f)


        t_x,t_y, t_z= clean_data( x_data_test, y_data_test, False , Mapping=Mapper)
                
        
        x_test.extend(t_x)
        y_test.extend(t_y)
        y_test_service.extend(t_z)

len(x_train), len(y_test)
            

in load from merged
loading from test files
found files :  5
home_os_final.json
home_sk_final.json
test_data_light.json
test_data_motion_2.json


(24173, 24560)

In [10]:
def predict_labels(classes,x_test_new,y_test_new,confidence):
    is_string=True
    print('predicting', classes)
    x_test_processed,y_test_processed_encoded, y_test_processed = pre_process_raw( x_test_new,
     y_test_new , dim_size, 
    test=True, normalize=False, classes=classes, string=is_string)
   
        
    def draw_xgb_tree():
        xgb = XGBClassifier()
        y = [labels.split(' ')[0] for labels in y_train_processed]
        # x_train_array = [map(int,x.split(' ')) for x in x_train_processed]      
          
        xgb.fit(x_train_processed, y)   
        # plot single tree
        plot_importance(xgb)
        plot_tree(xgb)
        plt.show()

    # draw_xgb_tree()
    def classify_xgb(x_test_processed):
        xgb_classifier = OneVsRestClassifier(XGBClassifier())
        if is_string:
            vectorizer = CountVectorizer(analyzer='word', ngram_range=(1,4))
            x_train_str = vectorizer.fit_transform(x_train_processed)            
            x_test_str = vectorizer.transform(x_test_processed)        
            xgb_classifier.fit(x_train_str, y_train_processed_encoded)

        else:
            xgb_classifier.fit(x_train_processed, y_train_processed_encoded)

        known_x_data=[]
        known_y_data=[]
        print( "==================HOME Case : Omid ============="  )
        rf_pred= xgb_classifier.predict_proba(x_test_str)
        #print(rf_pred)
#         __print_info(y_test_processed_encoded, rf_pred, classes, confidence)
        print(classes)
        print(len(classes))
        for ind,y_true in enumerate(y_test_processed):     
            print(y_true , rf_pred[ind])
            y_pred_class = [classes[i] for i,x in enumerate(rf_pred[ind]) if x>=confidence]
#             if 'known' not in classes: 
#                 print(y_true, y_pred_class)
#                 # if not y_true and y_pred_class and 'unknown' not in y_pred_class:
#                 if 'contact' in y_pred_class or 'contact' in y_true:
#                     print("True: ", y_true,"\n Pred: ",y_pred_class )
#                     # print("Last 5 true: ",y_test_processed[ind-10:ind])
#                     print(x_test_processed[ind])

            y_pred_class = y_pred_class if y_pred_class else ['none']
            if 'known' in classes and 'known' in y_pred_class:
                # print(x_test_processed[ind],"\nTrue: ",y_true, "\nPrediction: ", ' '.join(y_pred_class))
                known_x_data.append(x_test_processed[ind].split(' '))
                known_y_data.append(y_true.split(' '))
            else:
                known_x_data.append(x_test_processed[ind].split(' '))
                known_y_data.append(y_pred_class)
        # for item in zip(known_y_data,known_x_data):
        #     print(item)
        return known_x_data,known_y_data
    # classify_RandomForest()
    return classify_xgb(x_test_processed)


In [11]:

dim_size= 15
is_string =True
services_to_keep = ['unknown','known']

classes=None
terms_dic = {}


services_to_keep =['button-held',
 'button-pushed',
        "colorTemperature-XXX",
"contact-closed",
"contact-open",
"level-XXX",
"lock-locked",
"lock-unlocked",
"motion-active",
"motion-inactive",
"ping-ping",
"status-closed",
"status-open",
"switch-off",
"switch-on",
"temperature-XXX"
    ] 

services_to_keep= ['unknown','colorTemperature','level','switch','ping','temperature','motion' ,'lock','contact','status','button']
classes = sorted(services_to_keep)
x_train_processed,y_train_processed_encoded, y_train_processed = pre_process_raw( x_train, y_train , dim_size, test=False, normalize=False, classes=classes, string=is_string)

classes = sorted(services_to_keep)
if classes is None:
    classes  = sorted(list(np.unique(  np.concatenate( y_train  ))))
else :
    classes = sorted(classes)
if classes and 'unknown' not in classes:
    classes.append('unknown') 
    
xgb_x, xgb_y = predict_labels(classes,x_test,y_test,0.2)

['button', 'colorTemperature', 'contact', 'level', 'lock', 'motion', 'ping', 'status', 'switch', 'temperature', 'unknown']
(24173,)
predicting ['button', 'colorTemperature', 'contact', 'level', 'lock', 'motion', 'ping', 'status', 'switch', 'temperature', 'unknown']
['button', 'colorTemperature', 'contact', 'level', 'lock', 'motion', 'ping', 'status', 'switch', 'temperature', 'unknown']
(11835,)


/opt/anaconda3/envs/omid/lib/python3.6/site-packages/sklearn/multiclass.py:75: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
/opt/anaconda3/envs/omid/lib/python3.6/site-packages/sklearn/multiclass.py:75: UserWarning: Label not 1 is present in all training examples.
  str(classes[c]))
/opt/anaconda3/envs/omid/lib/python3.6/site-packages/sklearn/multiclass.py:75: UserWarning: Label not 2 is present in all training examples.
  str(classes[c]))
/opt/anaconda3/envs/omid/lib/python3.6/site-packages/sklearn/multiclass.py:75: UserWarning: Label not 3 is present in all training examples.
  str(classes[c]))
/opt/anaconda3/envs/omid/lib/python3.6/site-packages/sklearn/multiclass.py:75: UserWarning: Label not 4 is present in all training examples.
  str(classes[c]))
/opt/anaconda3/envs/omid/lib/python3.6/site-packages/sklearn/multiclass.py:75: UserWarning: Label not 5 is present in all training examples.
  str(classes[c]))
/opt/anaconda3/envs/omid/lib/python3.6/s

==================HOME Case : Omid =============
['button', 'colorTemperature', 'contact', 'level', 'lock', 'motion', 'ping', 'status', 'switch', 'temperature', 'unknown']
11
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 

unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown 

unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown 

...

unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown 

unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown 

unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown 

unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown 

unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown [0 0 0 0 0 0 0 0 0 0 1]
unknown 